In [115]:
import numpy as np
import matplotlib as plt
import sys
sys.path.insert(0, '..')
sys.path.insert(0, '../rainforest')
from paths import DATA_FOLDER
from data import get_class_data
from keras.applications.vgg16 import VGG16
from keras.layers import Dense
from keras.models import Model
from scipy import misc

In [2]:
train = get_class_data(train=True, label='mines')
validation = get_class_data(train=False, label='mines')

### Info

###### Validation:
    artisial mine: 95, conventional mine: 26, total: 12144
###### Train:
    artisial mine: 244, conventional mine: 74, total: 28335
    
    2 images in train set with both labels
    1 image in val set with both labels

In [132]:
# data generator
def data_generator_balanced(data, label, length, train):
    '''
    Input:
        data: pandas object containing the data 
        label: label of data you want to generate images from
        length: batch length
        train: if train folder or test
    '''
    data = data[['image_name', label]]
    cnt = 0
    folder = '/train-jpg/'
    if not train:
        folder = '/test-jpg/'
    while True:
        batch = np.zeros((length, 3, 256, 256))
        labels = np.ones((50))
        for i in range(length/2):  # get mine images
            img_name = np.random.choice(data[data[label] == 1]['image_name'].as_matrix())
            img = '..' + DATA_FOLDER + folder + img_name + '.jpg'
            batch[i, :, :, :] = misc.imread(img)[:,:,:3]
        for i in range(length/2, length):  # get non mine images
            img_name = np.random.choice(data[data[label] == 0]['image_name'].as_matrix())
            img = '..' + DATA_FOLDER + folder + img_name + '.jpg'
            batch[i, :, :, :] = misc.imread(img)[:,:,:3]
            labels[i] = 0
        
        # Shuffle batch
        idx = np.arange(length)
        np.random.shuffle(idx)
        batch = batch[idx]
        labels = labels[idx]
        
        yield batch, labels 

In [133]:
np.random.choice(train[train['conventional_mine'] == 1]['image_name'].as_matrix())

'train_17125'

In [134]:
gen = data_generator_balanced(train, 'conventional_mine', 30, True)
gen.next()

IOError: [Errno 2] No such file or directory: '../data/train-jpg/train_7119.jpg'

In [38]:
# augmentation generator
def augmentation_generator(data_gen):
    '''
    Input:
        data_gen: a data generator that generates batches of data that need to be augmented
    '''
    while True:
        yield None

In [9]:
def model():
    model = VGG16(include_top=False, weights='imagenet', input_shape=(3,256,256))
    
    x = model.output
    x = Dense(1024, activation='relu')(x)
    predictions = Dense(2, activation='softmax')(x)

    model = Model(inputs=model.input, outputs=predictions)
    
    for layer in model.layers:
        layer.trainable = False
    return model

In [10]:
a = model()
a.compile(optimizer='adam', loss='categorical_crossentropy')